In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

In [3]:
# Define the path to your CSV file and the chunk size
file_path = "C:/Users/HP/OneDrive/Desktop/Internship/xlsheets/aws_review_sofware_dataset.csv"
chunk_size = 25000

# Initialize an empty DataFrame to store the combined data
data = pd.DataFrame()
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    data = pd.concat([data, chunk], ignore_index=True)
    if len(data)>=25000: # Indentation fixed here
        data=data.iloc[:25000]
        break # Indentation fixed here; break should occur within the if block

# Now `data` contains the entire DataFrame
print(data.shape) 
print(data.head())

(25000, 13)
   Unnamed: 0  overall  verified   reviewTime      reviewerID        asin  \
0           0      4.0      True  03 11, 2014  A240ORQ2LF9LUI  0077613252   
1           1      4.0      True  02 23, 2014  A1YCCU0YRLS0FE  0077613252   
2           2      1.0      True  02 17, 2014  A1BJHRQDYVAY2J  0077613252   
3           3      3.0      True  02 17, 2014   APRDVZ6QBIQXT  0077613252   
4           4      5.0     False  10 14, 2013  A2JZTTBSLS1QXV  0077775473   

                        style         reviewerName  \
0  {'Format:': ' Loose Leaf'}           Michelle W   
1  {'Format:': ' Loose Leaf'}  Rosalind White Ames   
2  {'Format:': ' Loose Leaf'}       Allan R. Baker   
3  {'Format:': ' Loose Leaf'}                 Lucy   
4                         NaN            Albert V.   

                                          reviewText  \
0  The materials arrived early and were in excell...   
1  I am really enjoying this book with the worksh...   
2  IF YOU ARE TAKING THIS CLASS 

In [14]:
data.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,words,sentences
0,0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN,"[the, material, arrive, early, and, be, in, ex...",[The materials arrived early and were in excel...
1,1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN,"[i, be, really, enjoy, this, book, with, the, ...",[I am really enjoying this book with the works...
2,2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7,NaN,"[if, you, be, take, this, class, don, '', t, w...","[IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR..."
3,3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3,NaN,"[this, book, be, miss, page, !, !, !, importan...","[This book was missing pages!!!, Important pag..."
4,4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN,"[i, have, use, learnsmart, and, can, officiall...",[I have used LearnSmart and can officially say...


In [5]:
data.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [6]:
data["words"]="default value"
data["sentences"]="default value"


for i in range(data.shape[0]):
    data.at[i,"words"]= list("")
    data.at[i,"sentences"] = list("")

In [7]:
from nltk.tokenize import sent_tokenize

In [9]:
for i in range(499):
    l1= sent_tokenize(data.loc[i,"reviewText"])
    data.at[i,"sentences"]=l1

In [11]:
!pip install pywsd -q

In [10]:
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 7.936540126800537 secs.


In [12]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
for k in range(data.shape[0]):
    data.at[k,"words"]=list("")
    for j in range(len(data.loc[k,"sentences"])):
        data.at[k,"words"].extend(lemmatize_sentence(data.loc[k,"sentences"][j]))

In [16]:
data["words_sentences"] = "default"

In [18]:
import functools

for k in range(data.shape[0]):
    words_list = data.loc[k, "words"]
    
    if words_list:  # Check if the list is not empty
        data.loc[k, "words_sentences"] = functools.reduce(lambda a, b: str(a) + " " + str(b), words_list)
    else:
        data.loc[k, "words_sentences"] = ""  # Assign an empty string if the list is empty


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data.words_sentences)

In [29]:
data_x= pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [30]:
data_y=data["verified"]

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
L_encoder = LabelEncoder()

In [33]:
data_y=pd.DataFrame(L_encoder.fit_transform(data_y),columns=["verified"])

In [34]:
data_y.head()

,verified
0,1
1,1
2,1
3,1
4,0


## TRAIN TEST SPLIT

In [35]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3, random_state=42)

## GRADIENT BOOSTING CLASSIFIER

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
GBR = GradientBoostingClassifier(random_state=42)
GBR.fit(x_train, y_train)
y_pred=GBR.predict(x_test)
from sklearn.metrics import accuracy_score
training_score = GBR.score(x_train, y_train)*100
print(f'Training Score : {training_score}')
print("accuracy_score : ",accuracy_score(y_test,y_pred)*100)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score : 70.86857142857143
accuracy_score :  70.36


## RANDOMFOREST CLASSIFIER

In [39]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=42)
RF.fit(x_train, y_train)
y_pred=RF.predict(x_test)
from sklearn.metrics import accuracy_score
training_score = RF.score(x_train, y_train)*100
print(f'Training Score : {training_score}')
print("accuracy_score : ",accuracy_score(y_test,y_pred)*100)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Score : 70.90857142857143
accuracy_score :  70.52000000000001
